# Training and predicting with 3D U-Net

 ### 3D U-Net model implementtion

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate, LeakyReLU
import tensorflow as tf
import kera

In [1]:
def conv_block(input, num_filters):
    x = Conv3D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = LeakyReLU()(x)

    x = Conv3D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = LeakyReLU()(x)

    return x

#Encoder block: Conv block followed by maxpooling
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPooling3D((2, 2, 2))(x)
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation
def decoder_block(input, skip_features, num_filters):
    x = Conv3DTranspose(num_filters, (2, 2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

#Build Unet using the blocks
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:  #Binary
        activation = 'sigmoid'
    else:
        activation = 'softmax'

    outputs = Conv3D(n_classes, 1, padding="same", activation=activation)(d4)
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model

### Installation of patchify library

In [2]:
#Use patchify to break large volumes into smaller for training
!pip install patchify

### GPU availability 

In [4]:
#Make sure the GPU is available. 
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

In [4]:
from skimage import io
from patchify import patchify, unpatchify
import numpy as np
from matplotlib import pyplot as plt
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

### Images and masks loading

In [13]:
#Here we load 180x1024x1024 pixel volume. We will break it into patches of 15x64x64 for training with steps of 5x64x64. 
image = []

image.append(io.imread('/images/wt_pom1D_01_07_R3D_REF_image.tif'))
image.append(io.imread('/images/wt_pom1D_01_15_R3D_REF_image.tif'))
image.append(io.imread('/images/wt_pom1D_01_20_R3D_REF_image.tif'))
image.append(io.imread('/images/train/wt_pom1D_01_30_R3D_REF_image.tif'))

img_patches = []
img_patches.append(patchify(image[0], (16, 64, 64), step=(5, 64, 64)))
img_patches.append(patchify(image[1], (16, 64, 64), step=(5, 64, 64))) 
img_patches.append(patchify(image[2], (16, 64, 64), step=(5, 64, 64)))  
img_patches.append(patchify(image[3], (16, 64, 64), step=(5, 64, 64)))  

In [ ]:
mask = []

mask.append(io.imread('/masks/wt_pom1D_01_07_R3D_REF_mask.tif'))
mask.append(io.imread('/masks/wt_pom1D_01_15_R3D_REF_mask.tif'))
mask.append(io.imread('/masks/wt_pom1D_01_20_R3D_REF_mask.tif'))
mask.append(io.imread('/masks/wt_pom1D_01_30_R3D_REF_mask.tif'))

mask_patches = []
mask_patches.append(patchify(mask[0], (16, 64, 64), step=(5, 64, 64)))
mask_patches.append(patchify(mask[1], (16, 64, 64), step=(5, 64, 64)))  
mask_patches.append(patchify(mask[2], (16, 64, 64), step=(5, 64, 64)))  
mask_patches.append(patchify(mask[3], (16, 64, 64), step=(5, 64, 64)))  

### Reshape the inputs

In [ ]:
input_img = np.reshape(img_patches[0], (-1, img_patches[0].shape[3], img_patches[0].shape[4], img_patches[0].shape[5]))
input_mask = np.reshape(mask_patches[0], (-1, mask_patches[0].shape[3], mask_patches[0].shape[4], mask_patches[0].shape[5]))

for i in range(1, 4):
    input_img += np.reshape(img_patches[i], (-1, img_patches[i].shape[3], img_patches[i].shape[4], img_patches[i].shape[5]))
    input_mask += np.reshape(mask_patches[i], (-1, mask_patches[i].shape[3], mask_patches[i].shape[4], mask_patches[i].shape[5]))

input_img = np.array(input_img)
input_mask = np.array(input_mask)

print(input_img.shape)
print(input_mask.shape)

### Removing empy patches 

In [ ]:
idx_img = np.where(input_mask.mean(axis=(1,2,3)) != 0)[0]

input_img = input_img[idx_img]
input_mask = input_mask[idx_img]

print(input_img.shape[0])

### Standardize the input images and expand the dimension of the arrays by 1 (to match the input of the model)

In [22]:
train_img = input_img / input_img.max() 
train_img = np.expand_dims(train_img, axis=4)
train_mask = np.expand_dims(input_mask, axis=4)

### Binarize the masks 

In [2]:
train_mask[train_mask>1] = 1

NameError: name 'train_mask' is not defined

### One hot encoding of the masks

In [ ]:
n_classes=2
train_mask_cat = to_categorical(train_mask, num_classes=n_classes)

### Split randomly into training and validation set 

In [3]:
X_train, X_test, y_train, y_test = train_test_split(train_img, train_mask_cat, test_size = 0.20, random_state = 0)
print(X_train.shape)
print(X_test.shape)

NameError: name 'train_test_split' is not defined

### Define the dice coefficient 

In [ ]:
def dice_coefficient(y_true, y_pred):
    smoothing_factor = 1
    flat_y_true = K.flatten(y_true)
    flat_y_pred = K.flatten(y_pred)
    return (2. * K.sum(flat_y_true * flat_y_pred) + smoothing_factor) / (K.sum(flat_y_true) + K.sum(flat_y_pred) + smoothing_factor)

### Building our model 

In [ ]:
#Define parameters for our model.

patch_size1 = 16
patch_size2 = 64
patch_size3 = 64
channels=1

LR = 0.0001
optim = keras.optimizers.Adam(LR)

model = build_unet((patch_size1,patch_size2,patch_size3,channels), n_classes)

model.compile(optimizer = optim, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=dice_coefficient)
print(model.summary())

In [ ]:
print(model.input_shape)
print(X_train.shape)
print(model.output_shape)
print(y_train.shape)
print("-------------------")
print(X_train.max()) 

### Training the model

In [ ]:
#Fit the model
history=model.fit(X_train, 
          y_train,
          batch_size=8, 
          epochs=100,
          verbose=1,
          validation_data=(X_test, y_test))

### Saving the model for prediction

In [ ]:
#Save model for future use
model_path = '/saved_models/3dunetmodel_leaky_bs8_16x64x64_100epochs.h5'
model.save(model_path)

### Plotting the training and validation IoU and loss at each epoch 

In [4]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['dice_coefficient']
val_acc = history.history['val_dice_coefficient']

plt.plot(epochs, acc, 'y', label='Training Dice')
plt.plot(epochs, val_acc, 'r', label='Validation Dice')
plt.title('Training and validation Dice')
plt.xlabel('Epochs')
plt.ylabel('Dice')
plt.legend()
plt.show()

NameError: name 'history' is not defined

### Predict the model 

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
#Predict on the test data
y_pred_argmax=np.argmax(y_pred, axis=4)
y_test_argmax = np.argmax(y_test, axis=4)

In [ ]:
print(y_pred_argmax.shape)
print(y_test_argmax.shape)
print(np.unique(y_pred_argmax))

### Mean IoU 

In [ ]:
#Only works on TF > 2.0
from keras.metrics import MeanIoU
n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test_argmax, y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

### Testing random images 

In [ ]:
import random

test_img_number = random.randint(0, len(X_test)-1)
test_img = X_test[test_img_number]
ground_truth=y_test[test_img_number]

test_img_input=np.expand_dims(test_img, 0)


test_pred = my_model.predict(test_img_input)
test_prediction = np.argmax(test_pred, axis=4)[0,:,:,:]

ground_truth_argmax = np.argmax(ground_truth, axis=3)
print(ground_truth_argmax.shape)

#### Plotting the testing image, ground truth mask and the prediction 

In [ ]:
slice = random.randint(0, ground_truth_argmax.shape[0]-1)
plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[slice,:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth_argmax[slice,:,:])
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(test_prediction[slice,:,:])
plt.show()